## 得出 bigram + unigram 的次數集合

In [1]:
import re
from collections import Counter

# 只挑出文字不包含標點符號
def words(text): return re.findall(r'\w+', text.lower())


def sub_punctuation_mark(text):
    return re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+", " ",text.lower())

# 將[1:]和[:-1]zip再一起可以得到bigram
def create_bigrams(text): 
    bigrams = []
    for sentence in text.split('\n'):
        sentence = sub_punctuation_mark(sentence).strip()
        if sentence.split(" ")[:-1] != '' and sentence.split(" ")[1:] != '':
            for b in zip(sentence.split(" ")[:-1], sentence.split(" ")[1:]):
                 bigrams.append(b)
    return bigrams

BIGRAMS = Counter(create_bigrams(open('big.txt').read()))           
WORDS = Counter(words(open('big.txt').read()))
BIGRAMS_WORDS = BIGRAMS + WORDS

In [2]:
print(len(BIGRAMS))
print(len(WORDS))

355138
32198


## 得出所有ipnut所有組合，並選用known中出現機率最高的更正選項

In [4]:
def P(word, N=sum(BIGRAMS_WORDS.values())): 
    "Probability of `word`."
    return BIGRAMS_WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

# bigram 第一個、第二個字各 edit 一次
def bigram_edits1(t):
    tuple_set = set()
    for first in edits1(t[0]):
        for second in edits1(t[1]):
            tuple_set.add((first,second))
    return tuple_set

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in BIGRAMS_WORDS)

def candidates(word):
    "Generate possible spelling corrections for word."
    split_word = word.split(' ')
    if len(split_word)==1:
        return (known([word]) or known(edits1(word)) or [word])
    else:
        word = (split_word[0],split_word[1])
        ed1 = known(bigram_edits1(word))
        return (known([word]) or ed1 or [word])

In [5]:
correction('prepareq my')

('prepared', 'by')

In [10]:
correction('see yo')

('seem', 'to')